In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from collections import defaultdict
import numpy as np
from numpy import unique
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import folium
from folium.plugins import FastMarkerCluster
from geopy.extra.rate_limiter import RateLimiter
from sklearn.cluster import KMeans

# LOAD DATA and PREPROCESSING

In [ ]:
data0 = pd.read_csv('D:/dataset/Flat_resale 2.csv')
data = pd.read_csv('D:/dataset/flat_resale.csv')
data = pd.concat([data0, data], sort=False)

In [ ]:
data.head(3)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
dataframe = data.copy()

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['month']) 
dataframe['month'] = dataframe['date'].apply(lambda date:date.month) 
dataframe['year'] = dataframe['date'].apply(lambda date:date.year)

In [ ]:
dataframe.head(2)

In [ ]:
cpi=pd.read_excel('D:/dataset/CPI_Sing.xlsx')
cpi['date'] = pd.to_datetime(cpi['date'], format='%Y %b') # to datetime

In [ ]:
dataframe = dataframe.merge(cpi, on='date', how='left')

In [ ]:
dataframe['real_price'] = round((dataframe['resale_price'] / dataframe['cpi']) * 100,0)

In [ ]:
dataframe['storey_range'] = dataframe['storey_range'].apply(lambda storey_range:storey_range[:2])

In [ ]:
dataframe.head()

In [ ]:
#Geocode by town (Singapore is so small that geocoding by addresses might not make much difference compared to geocoding to town)
#tạo 2 cột tung độ và vĩ độ dựa trên Town
town = [x for x in dataframe['town'].unique().tolist() 
            if type(x) == str]
latitude = []
longitude =  []
for i in range(0, len(town)):
    # remove things that does not seem usefull here
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        loc = geolocator.geocode(town[i])
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
        #print('The geographical coordinate of location are {}, {}.'.format(loc.latitude, loc.longitude))
    except:
        # in the case the geolocator does not work, then add nan element to list
        # to keep the right size
        latitude.append(np.nan)
        longitude.append(np.nan)
# create a dataframe with the locatio, latitude and longitude
df_ = pd.DataFrame({'town':town, 
                    'latitude': latitude,
                    'longitude':longitude})
# merge on Restaurant_Location with rest_df to get the column 
dataframe = dataframe.merge(df_, on='town', how='left')

In [ ]:
flat_typeDict = {'1 ROOM': 1,'2 ROOM': 2,'3 ROOM': 3,'4 ROOM': 4,'5 ROOM': 5,'EXECUTIVE': 6,'MULTI-GENERATION': 7,}
dataframe['flat_type'] = dataframe['flat_type'].replace(flat_typeDict, regex=True)

In [ ]:
location = dataframe[['latitude','longitude','real_price']]
location.head()

In [ ]:
mmscale = MinMaxScaler()
mmscale.fit(location)

In [ ]:
location_scaled = pd.DataFrame(mmscale.transform(location), columns = ['latitude', 'longitude','real_price'])
location_scaled.head()

In [ ]:
location_scaled.dtypes

# sử dụng KMEANS để phân các Town vào các cụm theo giá

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(location_scaled)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
km_final = KMeans(n_clusters = 5, random_state = 123)
km_final.fit(location_scaled)
location_scaled['Cluster'] = km_final.labels_

In [ ]:
location_scaled

In [ ]:
dataframe['Cluster'] = km_final.labels_

In [ ]:
dataframe=dataframe.drop('town',axis='columns')
dataframe.head()

In [ ]:
corrmat = dataframe.corr()
#resale_price correlation matrix
k = 12 #number of variables for heatmap
cols = corrmat.nlargest(k, 'resale_price')['resale_price'].index
cm = np.corrcoef(dataframe[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
dataframe = dataframe.drop('block',axis=1)
dataframe = dataframe.drop('month',axis=1)
dataframe = dataframe.drop('street_name',axis=1)
dataframe = dataframe.drop('flat_model',axis=1)
dataframe = dataframe.drop('year',axis=1)
dataframe = dataframe.drop('date',axis=1)
dataframe = dataframe.drop('remaining_lease',axis=1)
dataframe = dataframe.drop('cpi',axis=1)
dataframe = dataframe.drop('resale_price',axis=1)
dataframe = dataframe.drop('timeseries',axis=1)


In [ ]:
dataframe.head()

In [ ]:
dataframe['real_price'] = round((dataframe['real_price'] / 1000))

In [ ]:
dataframe

In [ ]:
dataframe=dataframe.astype('float')

# FINAL DATA

In [ ]:
dataframe

In [ ]:
dataframe.to_csv("D:/dataset/Flat_PreProcessed.csv",index=False)

# PREPROCESSING DATA FROM ANOTHER WEB


In [ ]:
datatest = pd.read_excel('D:/dataset/testdata.xlsx')
datatest.head()

In [ ]:
datatest['date'] = pd.to_datetime(datatest['date'])

In [ ]:
datatest = datatest.merge(cpi, on='date', how='left')

In [ ]:
datatest['real_price'] = round((datatest['Price'] / datatest['cpi']) * 100,0)

In [ ]:
datatest['real_price'] = round((datatest['real_price'] / 1000))

In [ ]:
flat_typeDict = {'1 ROOM': 1,'2 ROOM': 2,'3 ROOM': 3,'4 ROOM': 4,'5 ROOM': 5,'EXECUTIVE': 6,'MULTI-GENERATION': 7,}
datatest['flat_type'] = datatest['flat_type'].replace(flat_typeDict, regex=True)
datatest = datatest.merge(df_, on='town', how='left')

In [ ]:
datatest = datatest.drop(['date','timeseries','cpi'],axis =1)

In [ ]:
lct = location_scaled.drop('real_price', axis=1)

In [ ]:
lct['Cluster'] = lct.groupby('latitude')['Cluster'].transform(lambda x: x.mode()[0])

In [ ]:
lct.drop_duplicates(inplace=True)

In [ ]:
lct = lct.reset_index(drop=True)

In [ ]:
lct = lct.drop(['latitude','longitude'],axis=1)

In [ ]:
# Min-Max Normalization
df = df_.drop('town', axis=1)
df_norm = (df-df.min())/(df.max()-df.min())
df_norm = pd.concat((df_.town,df_norm), 1)

In [ ]:
group =  pd.concat([df_norm,lct],axis =1)


In [ ]:
group = group.drop(['latitude','longitude'],axis=1)

In [ ]:
datatest1 = datatest.merge(group, on='town', how='left')

In [ ]:
datatest1 = datatest1.drop(['Price','town'],axis=1)


In [ ]:
datatest1.head()

In [ ]:
datatest1=datatest1.astype('float')

In [ ]:
datatest1.head()

In [ ]:
datatest1.to_csv("D:/dataset/Flat_test_PreProcessed.csv",index=False)